<a href="https://colab.research.google.com/github/churamani2030dev/proj3_DS_coding/blob/main/proj3_DS_coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⬇️ Get the project

In [30]:
!git clone https://github.com/udacity/dsnd-dashboard-project
%cd dsnd-dashboard-project

Cloning into 'dsnd-dashboard-project'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 160 (delta 47), reused 38 (delta 38), pack-reused 78 (from 1)
Receiving objects: 100% (160/160), 258.78 KiB | 1.37 MiB/s, done.
Resolving deltas: 100% (59/59), done.
/content/dsnd-dashboard-project/dsnd-dashboard-project/dsnd-dashboard-project


# 🔧 Install deps (project + package)

In [32]:
!pip install -q -r requirements.txt


# (Nice to have)

In [18]:
!pip install -q pytest pyngrok

In [20]:
# Add both the project root and the package folder to sys.path
import sys, pathlib, os
ROOT = pathlib.Path.cwd()
PKG  = ROOT / "python-package"
sys.path += [str(ROOT), str(PKG)]


# Verify key files exist
for p in [
    "python-package/employee_events/employee_events.db",
    "assets/model.pkl",
    "report/dashboard.py",
    "tests/test_employee_events.py"
]:
    print(p, "✅" if (ROOT / p).exists() else "❌")

python-package/employee_events/employee_events.db ✅
assets/model.pkl ✅
report/dashboard.py ✅
tests/test_employee_events.py ✅


In [21]:
%%writefile python-package/employee_events/sql_execution.py
from pathlib import Path
import sqlite3, pandas as pd

DB_PATH = Path(__file__).resolve().parent / "employee_events.db"

def run(query: str, params: dict | tuple = ()) -> pd.DataFrame:
    with sqlite3.connect(DB_PATH) as conn:
        conn.row_factory = sqlite3.Row
        cur = conn.execute(query, params if isinstance(params, tuple) else dict(params))
        rows = cur.fetchall()
    return pd.DataFrame([dict(r) for r in rows])

Overwriting python-package/employee_events/sql_execution.py


In [22]:
%%writefile python-package/employee_events/query_base.py
from .sql_execution import run

def employee_timeseries(employee_id: int):
    q = """
    SELECT event_date,
           SUM(positive_events) AS pos,
           SUM(negative_events) AS neg
    FROM employee_events
    WHERE employee_id = :eid
    GROUP BY event_date
    ORDER BY event_date;
    """
    return run(q, {"eid": employee_id})

def team_summary(team_id: int):
    q = """
    SELECT e.employee_id,
           e.first_name || ' ' || e.last_name AS name,
           SUM(ev.positive_events) AS pos,
           SUM(ev.negative_events) AS neg
    FROM employee e
    JOIN employee_events ev USING(employee_id)
    WHERE e.team_id = :tid
    GROUP BY e.employee_id, name
    ORDER BY pos DESC;
    """
    return run(q, {"tid": team_id})

Overwriting python-package/employee_events/query_base.py


In [23]:
%%writefile python-package/employee_events/employee.py
import numpy as np, pandas as pd
from .query_base import employee_timeseries

def get_employee_timeseries(entity_id, model):
    return employee_timeseries(int(entity_id))

def get_employee_snapshot(entity_id, model):
    ts = employee_timeseries(int(entity_id))
    pos30 = ts.tail(30)["pos"].sum() if not ts.empty else 0
    neg30 = ts.tail(30)["neg"].sum() if not ts.empty else 0
    # Example feature vector; adjust to what your model expects
    X = pd.DataFrame([{"pos30": pos30, "neg30": neg30}])
    try:
        risk = float(model.predict_proba(X)[:,1])
    except Exception:
        risk = 0.0
    return {"pos30": int(pos30), "neg30": int(neg30), "risk": risk}

Overwriting python-package/employee_events/employee.py


In [24]:
%%writefile python-package/employee_events/team.py
from .query_base import team_summary

def get_team_table(entity_id, model):
    return team_summary(int(entity_id))

Overwriting python-package/employee_events/team.py


In [25]:
%%writefile report/utils.py
import pickle, pathlib
ASSETS = pathlib.Path(__file__).resolve().parents[1] / "assets"

_model = None
def get_model():
    global _model
    if _model is None:
        with open(ASSETS / "model.pkl", "rb") as f:
            _model = pickle.load(f)
    return _model

Overwriting report/utils.py


In [26]:
%%writefile report/dashboard.py
from report.utils import get_model
from employee_events.employee import get_employee_snapshot, get_employee_timeseries
from employee_events.team import get_team_table

# If your FastHTML skeleton exposes classes, import them here.
# Below is a simple bottle/fastapi-like sketch; adapt to your framework.

import http.server, socketserver, json

PORT = 8000
MODEL = get_model()

class Handler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        if self.path.startswith("/api/employee/"):
            eid = int(self.path.split("/")[-1])
            snap = get_employee_snapshot(eid, MODEL)
            self._send_json(snap)
        else:
            # Serve a tiny HTML that calls the API (placeholder)
            html = """
            <html><body>
              <h2>Employee Dashboard</h2>
              <p>Try /api/employee/1</p>
            </body></html>
            """
            self._send_html(html)

    def _send_json(self, obj):
        data = json.dumps(obj).encode()
        self.send_response(200)
        self.send_header("Content-Type","application/json")
        self.send_header("Content-Length", str(len(data)))
        self.end_headers()
        self.wfile.write(data)

    def _send_html(self, html):
        data = html.encode()
        self.send_response(200)
        self.send_header("Content-Type","text/html")
        self.send_header("Content-Length", str(len(data)))
        self.end_headers()
        self.wfile.write(data)

if __name__ == "__main__":
    with socketserver.TCPServer(("", PORT), Handler) as httpd:
        print(f"Serving on http://127.0.0.1:{PORT}")
        httpd.serve_forever()

Overwriting report/dashboard.py


In [27]:
!pytest -q


no tests ran in 0.02s


In [28]:
# run your dashboard (use the project root as CWD)
!PYTHONPATH=$PYTHONPATH:. python report/dashboard.py

Traceback (most recent call last):
  File "/content/dsnd-dashboard-project/dsnd-dashboard-project/report/dashboard.py", line 2, in <module>
    from employee_events.employee import get_employee_snapshot, get_employee_timeseries
ModuleNotFoundError: No module named 'employee_events'
